In [1]:
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F

In [4]:
image_loc = '/Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/'

In [5]:
image_loc += "data/final_dataset/final/"

In [28]:
tens = torch.randn((384, 64,124,124))

In [29]:
tens.shape

torch.Size([384, 64, 124, 124])

In [26]:
class encodedFloatItem(ItemBase):
    "Basic class for float items."
    def __init__(self,obj): self.data,self.obj = np.array(obj).astype(np.float32),obj
    def __str__(self):  return str(self.data.shape)
    def __hash__(self): return hash(str(self))



In [27]:
class encodedList(FloatList):
    def __init__(self, items:Iterator, log:bool=False, **kwargs):
        if isinstance(items, ItemList):
            items = items.items
        super(FloatList,self).__init__(items,**kwargs)
    def get(self,i):
        return encodedFloatItem(torch.Tensor(super(FloatList,self).get(i).astype('float32')))

In [30]:
dat = encodedList(tens).split_none()

In [31]:
dat

ItemLists;

Train: encodedList (384 items)
(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124)
Path: .;

Valid: encodedList (0 items)

Path: .;

Test: None

In [33]:
tens2 = torch.randn((96, 64,124,124))

In [34]:
dat.valid = encodedList(tens2)

In [35]:
dat = dat.label_from_lists(dat.train,dat.valid)

In [36]:
dat

LabelLists;

Train: LabelList (384 items)
x: encodedList
(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124)
y: ItemList
(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124)
Path: .;

Valid: LabelList (96 items)
x: encodedList
(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124)
y: ItemList
(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124),(64, 124, 124)
Path: .;

Test: None

In [39]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(64, 112, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor = 2, mode = 'bilinear'),
            nn.Conv2d(112, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(inplace=True)
        )
        
    def encode(self,x): return self.encoder(x)
    
    def decode(self,x): return torch.clamp(self.decoder(x), min = 0, max=1)
        

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return torch.clamp(decoded, min=0, max=1)

In [40]:
ae = AutoEncoder()

In [50]:
dat.path = image_loc

In [55]:
dat = dat.databunch()

In [56]:
learn = Learner(dat, ae, loss_func=F.mse_loss)

In [57]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


KeyboardInterrupt: 

In [32]:
32*3

96

In [25]:
dat = ItemList(data).split_none()

KeyboardInterrupt: 

In [20]:
size = 224
batchsize = 32
tfms = get_transforms(do_flip = False)
src = (ImageImageList.from_folder(image_loc).split_by_rand_pct(seed=2).label_from_func(lambda x: x))
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats))#, do_y = False))

In [21]:
data

ImageDataBunch;

Train: LabelList (444 items)
x: ImageImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
Path: /Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/final_dataset/final;

Valid: LabelList (110 items)
x: ImageImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
Path: /Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/final_dataset/final;

Test: None